# Trenowanie modelu

Wybrałam zadanie Emotion and sentiment recognition. W tym notatniku opisuję trenowanie modeli i pokazuję ich wyniki. 

## Przygotowanie do pracy

Ponieważ trenowałam na własnym komputerze a nie na Colabie, to upewniam się że karta graficzna jest dostępna i ustawiam ją jako domyślnie używane urządzenie:

In [1]:
import torch

if torch.cuda.is_available():       
    print(f'GPU {torch.cuda.get_device_name(0)} will be used.')
    device = torch.device("cuda")
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

GPU NVIDIA GeForce GTX 1660 Ti will be used.


Usuwam cache, żeby podczas trenowania nie dostawać błędu CUDA_out_of_memory:

In [2]:
import gc
gc.collect()
torch.cuda.empty_cache()

## Przygotowanie danych

Wcześniej sklonowałam z githuba repozytorium z danymi, wczytuję odpowiednie pliki tutaj:

In [3]:
import pandas as pd

with open('2024-emotion-recognition/train/in.tsv', encoding='utf-8') as file:
  X_train = pd.read_csv(file, sep='\t')

with open('2024-emotion-recognition/train/expected.tsv', encoding='utf-8') as file:
  Y_train = pd.read_csv(file, sep='\t')
  Y_train = Y_train.astype(float)

sentences = X_train.text.values
labels = Y_train.values

Zapisuję nazwy etykiet i przypisuję im ID, żeby przekazać potem te informacje modelowi:

In [4]:
colnames = Y_train.columns
id2label = {}
label2id = {}
for i, name in enumerate(colnames):
    id2label[i] = name
    label2id[name] = i

Ładuję tokenizer od modelu twitter-sentiment-pl-base (https://huggingface.co/bardsai/twitter-sentiment-pl-base). Jest to model do analizy sentymentu, który znalazłam na HuggingFace, był trenowany na danych z twittera z TweetEval (Barbieri i in. 2020, link: https://aclanthology.org/2020.findings-emnlp.148.pdf) przetłumaczonych na język polski. Wydaje mi się, że te dane treningowe są dość podobne do tego co jest przetwarzane w tym projekcie, więc liczę że to pozytywnie wpłynie na wyniki modelu.

In [5]:
# %pip install -U sacremoses # ta paczka była potrzebna do używania HerbertTokenizer
from transformers import HerbertTokenizer

tokenizer = HerbertTokenizer.from_pretrained('bardsai/twitter-sentiment-pl-base', do_lower_case=True)

c:\Users\magda\kogni\inzynieria lingwistyczna\zaliczenie\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sprawdzam maksymalną długość zdania w zbiorze danych, ale trenowanie na tak długich inputach zajmuje zbyt dużo czasu, więc później ograniczyłam to do 50.

In [6]:
max_len = 0

for sent in sentences:
    input_ids = tokenizer.encode(sent)
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  397


Robię encoding dla zdań w zestawie treningowym:

In [7]:
from torch.utils.data import TensorDataset

input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
        sent,
        add_special_tokens = True,
        max_length = 50,
        truncation = True,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

dataset = TensorDataset(input_ids, attention_masks, labels) # łączę encodingi zdań, attention masks i etykiety w dataset

c:\Users\magda\kogni\inzynieria lingwistyczna\zaliczenie\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Ładuję zbiory danych do DataLoaderów, które pozwalają na iterowanie po kolejnych porcjach danych (o wielkości sprecyzowanej przez parametr batch_size). DataLoader do danych treningowych wybiera próbki w losowej kolejności, w przypadku danych walidacyjnych nie jest to konieczne.

In [8]:
from torch.utils.data import random_split, DataLoader, RandomSampler, SequentialSampler

def split_and_load(dataset, train_split, batch_size):
    train_size = int(train_split * len(dataset))
    val_size = len(dataset) - train_size

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset),
                batch_size = batch_size
            )

    validation_dataloader = DataLoader(
                val_dataset,
                sampler = SequentialSampler(val_dataset),
                batch_size = batch_size
            )
    
    return train_dataloader, validation_dataloader

## Definicje funkcji

Ładuję optimizer:

In [9]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

def optimizer_setup(model, epochs, lr, train_dataloader):
    optimizer = AdamW(model.parameters(), lr = lr)

    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)
    return optimizer, scheduler

Definiuję funkcję do liczenia accuracy i f1:

In [10]:
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix, f1_score

np.seterr(divide='ignore', invalid='ignore') # because division by zero can happen in f1_per_label

def flat_accuracy(preds, labels):
    preds = np.array(torch.round(torch.sigmoid(torch.tensor(preds))))
    labels = np.array(labels)
    correct = 0
    for i in range(len(preds)):
        if np.array_equal(preds[i], labels[i]):
            correct += 1
    return correct/len(preds)

def f1_general(preds, labels):
    preds = np.array(torch.round(torch.sigmoid(torch.tensor(preds))))
    labels = np.array(labels)

    result = f1_score(y_true=labels, y_pred=preds, average='macro', zero_division=0)
    return result

def f1_per_label(preds, labels):
    t_preds = torch.round(torch.sigmoid(torch.tensor(preds)))
    conf_matrices = multilabel_confusion_matrix(labels, t_preds) # list of matrices for all labels
    # tn = conf_matrices[:, 0, 0] # not used
    tp = conf_matrices[:, 1, 1]
    fn = conf_matrices[:, 1, 0]
    fp = conf_matrices[:, 0, 1]

    f1 = np.nan_to_num(2*tp / (2*tp + fn + fp), nan=1) # array of f1 scores for all labels
    # prec = np.nan_to_num(tp/(tp+fp), nan=1)
    # rec = np.nan_to_num(tp/(tp+fn), nan=1)
    # f1 = 2*prec*rec/(prec+rec)

    return f1

Dodatkowo definiuję funkcję, która pozwala ładnie printować czas trenowania:

In [11]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

Definiuję funkcję trenującą, która dostaje model, dane i parametry trenowania:

In [12]:
import wandb
import datetime

def training(model, dataset, hyperparameters):
    (lr, epochs, batch_size) = hyperparameters

    train_dataloader, validation_dataloader = split_and_load(dataset, 
                                                             train_split=0.8, 
                                                             batch_size=batch_size)
    optimizer, scheduler = optimizer_setup(model=model, 
                                           epochs=epochs, 
                                           lr=lr, 
                                           train_dataloader=train_dataloader)

    wandb.init(
        project='lingen',
        config={
            'dataset': '2024-emotion-recognition',
            'learning_rate': lr,
            'epochs': epochs,
            'batch_size': batch_size
        }
    )

    print(f'\ntraining in {epochs} epochs with learning rate = {lr} and batch size = {batch_size}\n')

    training_stats = []
    total_t0 = time.time()

    model_name = 'model-' + datetime.datetime.now().strftime('%d-%f')

    for e in range(epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(e + 1, epochs))
        print('Training...')
        t0 = time.time()
        total_train_loss = 0 # resetting the loss for this epoch
        all_train_acc = []

        model.train()

        for step, batch in enumerate(train_dataloader):

            # printing information about progress:
            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            
            # unpacking the training batch to the gpu:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # clearing gradients:
            model.zero_grad()

            # a forward pass -- predicting labels for given inputs
            fwd = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
            
            loss = fwd['loss']
            logits = fwd['logits']

            total_train_loss += loss.item() # saving the loss for metrics
            loss.backward() # a backward pass to calculate the gradients

            # Move logits and labels to CPU
            cpu_logits = logits.detach().cpu().numpy()
            cpu_label_ids = b_labels.to('cpu').numpy()
            all_train_acc.append(flat_accuracy(cpu_logits, cpu_label_ids))

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # update the parameters
            optimizer.step()
            scheduler.step()
            
        avg_train_loss = total_train_loss / len(train_dataloader)       
        avg_train_acc = np.mean(np.array(all_train_acc))     
    
        training_time = format_time(time.time() - t0)
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))

        print("")
        print("Running Validation...\n")
        t0 = time.time()

        model.eval()

        total_eval_accuracy = 0
        total_eval_loss = 0
        total_acc_per_lab = []
        total_f1 = []

        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # no need to calculate gradients for evaluation
            with torch.no_grad(): 
                fwd = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            
            loss = fwd['loss']
            logits = fwd['logits']
                
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            total_eval_accuracy += flat_accuracy(logits, label_ids)
            total_acc_per_lab.append(f1_per_label(logits, label_ids))
            total_f1.append(f1_general(logits, label_ids))

        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        avg_label_acc = np.mean(np.array(total_acc_per_lab), axis=0)
        label_dict = {colnames[i]: avg_label_acc[i] for i in range(len(colnames))}
        avg_f1 = np.mean(np.array(total_f1), axis=0)
        validation_time = format_time(time.time() - t0)
        print('average f1 score for every label:')
        print(pd.DataFrame(label_dict,
                           index=[0]))
        
        # print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
        # print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("\nValidation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        current_stats = {
                'epoch': e + 1,
                'Training Loss': avg_train_loss,
                'Training Accur.': avg_train_acc,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Valid. F1': avg_f1
            }
        # record statistics in wandb
        wandb.log(current_stats)
        wandb.log(label_dict)

        current_stats['Training Time'] = training_time
        current_stats['Validation Time'] = validation_time
        training_stats.append(current_stats)

    wandb.finish()
    print("")
    print("Training complete!")
    print("Total training took {:} (h:mm:ss)\n".format(format_time(time.time()-total_t0)))

    return model_name, training_stats

## Trenowanie

Przygotowuję listę, w której spiszę parametry trenowania i wyniki modelu oraz słownik, w którym będę przechowywać modele:

In [13]:
grid_rows = []
models = {}

Wybieram parametry przy jakich będę trenować:

In [24]:
from itertools import product

LR = [4e-05, 4e-03]
epochs = [6]
batch_size = [32, 128]
hyperparameters = list(product(LR, epochs, batch_size))
print(hyperparameters)
hp = hyperparameters[3]

[(4e-05, 6, 32), (4e-05, 6, 128), (0.004, 6, 32), (0.004, 6, 128)]


Ładuję model i wandb, trenuję, spisuję parametry i wyniki. Po wytrenowaniu jednego modelu zmieniam w bloku powyżej której konfiguracji parametrów chcę użyć i włączam poniższy blok jeszcze raz. Miałam tutaj wcześniej pętlę która przechodziła przez wszystkie konfiguracje parametrów, ale czas trenowania się zwiększał z każdą kolejną iteracją i zrezygnowałam z tego rozwiązania.

Używam BertForSequenceClassification, więc model ma dodatkową warstwę do klasyfikacji sentymentu, a przy ładowaniu modelu definiuję ile etykiet ma przypisywać ostatnia warstwa (num_labels) i jakie to są etykiety (id2label i label2id).

In [25]:
import pandas as pd
from transformers import BertForSequenceClassification
import gc

gc.collect()
torch.cuda.empty_cache()

print('\nloading the model...\n')
model = BertForSequenceClassification.from_pretrained(
    "bardsai/twitter-sentiment-pl-base",
    problem_type='multi_label_classification',
    num_labels = len(colnames),
    id2label = id2label,
    label2id = label2id,
    ignore_mismatched_sizes=True # bo normalnie ten model ma 3 etykiety w outpucie, a mój ma 11
)

model.cuda() # żeby model działał na karcie graficznej

model_name, training_stats = training(model=model,
                                        dataset=dataset,
                                        hyperparameters=hp)

df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
print('\ntraining stats from each epoch:\n')
print(df_stats)

grid_rows.append([model_name, hp[0], hp[1], hp[2], training_stats[-1]['Valid. F1'], training_stats[-1]['Valid. Accur.'], training_stats[-1]['Valid. Loss']])
models[model_name] = model


loading the model...



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bardsai/twitter-sentiment-pl-base and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([11]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([11, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



training in 6 epochs with learning rate = 0.004 and batch size = 128


======== Epoch 1 / 6 ========
Training...
  Batch    40  of     45.    Elapsed: 0:11:42.

  Average training loss: 0.56
  Training epoch took: 0:12:55

Running Validation...

average f1 score for every label:
   Joy  Trust  Anticipation  Surprise  Fear  Sadness  Disgust  Anger  \
0  0.0    0.0           0.0       0.0   0.0      0.0      0.0    0.0   

   Positive  Negative  Neutral  
0       0.0       0.0      0.0  

Validation took: 0:00:21

======== Epoch 2 / 6 ========
Training...
  Batch    40  of     45.    Elapsed: 0:11:24.

  Average training loss: 0.52
  Training epoch took: 0:12:44

Running Validation...

average f1 score for every label:
        Joy  Trust  Anticipation  Surprise  Fear  Sadness  Disgust  Anger  \
0  0.645387    0.0           0.0       0.0   0.0      0.0      0.0    0.0   

   Positive  Negative  Neutral  
0  0.686032       0.0      0.0  

Validation took: 0:00:20

======== Epoch 3 / 6 ===

Anger,▁▁▁▁▁▁
Anticipation,▁▁▁▁▁▁
Disgust,▁▁▁▁▁▁
Fear,▁▁▁▁▁▁
Joy,▁█▁▁█▁
Negative,▁▁▁▁▁▁
Neutral,▁▁▁▁▁▁
Positive,▁█████
Sadness,▁▁▁▁▁▁
Surprise,▁▁▁▁▁▁
Training Accur.,▁▂▄█▅▂



Training complete!
Total training took 1:10:37 (h:mm:ss)


training stats from each epoch:

       Training Loss  Training Accur.  Valid. Loss  Valid. Accur.  Valid. F1  \
epoch                                                                          
1           0.556638         0.032981     0.522146        0.00000   0.000000   
2           0.518686         0.034849     0.517056        0.12475   0.121038   
3           0.512783         0.040625     0.512276        0.00000   0.062367   
4           0.511396         0.050342     0.511930        0.00000   0.062367   
5           0.510574         0.042998     0.514921        0.12475   0.121038   
6           0.508579         0.034844     0.511382        0.00000   0.062367   

      Training Time Validation Time  
epoch                                
1           0:12:55         0:00:21  
2           0:12:44         0:00:20  
3           0:10:54         0:00:17  
4           0:09:51         0:00:21  
5           0:11:22         0:00:20  


## Zakończenie pracy

Po wytrenowaniu modeli tworzę tabelkę z ich wynikami:

In [26]:
grid_table = pd.DataFrame(grid_rows, columns=['model name', 'LR', 'epochs', 'batch size', 'f1 score', 'accuracy', 'loss'])
print('\nAll models trained, stats for each model:\n')
grid_table


All models trained, stats for each model:



,model name,LR,epochs,batch size,f1 score,accuracy,loss
0,model-16-441101,0.00004,6,32,0.534445,0.355128,0.331367
1,model-16-136630,0.00004,6,128,0.499651,0.323167,0.346022
2,model-16-592467,0.00400,6,32,0.000000,0.000000,0.508371
3,model-16-032198,0.00400,6,128,0.062367,0.000000,0.511382


Zapisuję najlepszy (lub wybrany) model:

In [17]:
import os

best_model = grid_table[grid_table.accuracy == max(grid_table['accuracy'])]['model name'].item()
# best_model = 'model-15-306151'
output_dir = f'./saved_models/{best_model}'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model = models[best_model]

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)

grid_table.to_csv(output_dir + '/gridsearch.csv')

Saving model to ./saved_models/model-16-441101


***
# Wyniki

Robiłam kilka podejść do trenowania, ten notatnik zmieniał się z każdym kolejnym, więc nie wszystkie wykresy tworzyłam od początku.

Te wykresy wyszły za pierwszym podejściem, przy którym jeszcze się orientowałam jak działa wandb:

![1st-attempt](wyniki/1st-attempt.png)

Tutaj trenowałam przy parametrach widocznych poniżej, nie dokończyłam tego podejścia, bo zobaczyłam jak dużo czasu mi to jeszcze zajmie a chciałam trochę przeorganizować kod i dodać więcej metryk.

![parameters1](wyniki/parameters1.png)

Przy drugim podejściu dodałam więcej metryk i spróbowałam trochę innych parametrów -- wyniki wyszły trochę lepsze, ale nie bardzo dużo. Tutaj nadal miałam ustawione trenowanie w pętli, więc ten czas znowu bardzo wzrósł z kolejnymi iteracjami i później zrezygnowałam z pętli. 

![parameters2](wyniki/parameters2.png)

![2nd-attempt](wyniki/2nd-attempt.png)

Przy trzecim podejściu postanowiłam zmieniać tylko batch_size i przy mniejszych wartościach wyszło mi nieco wyższe niż wcześniej accuracy.

![parameters3](wyniki/parameters3.png)

![3rd-attempt](wyniki/3rd-attempt.png)

Po drugim podejściu doszłam do wniosku, że chciałabym zobaczyć jeszcze jak zmieniały się f1 score dla każdej etykiety. Najciekawszą rzeczą wydało mi się tutaj, że dla etykiet Fear i Surprise wynik był bardzo podobny przez cały czas trenowania, ale bardzo różny dla każdego z modeli.

![3rd-attempt-emotions1](wyniki/3rd-attempt-emotions1.png)
![3rd-attempt-emotions2](wyniki/3rd-attempt-emotions2.png)

Ostatnie podejście było bardzo słabe, zamieszczam wykresy ale nie zapisałam z niego żadnego modelu.

![parameters4](wyniki\parameters4.png)

![4th-attempt](wyniki\4th-attempt.png)

![4th-attempt-emotions1](wyniki\4th-attempt-emotions1.png)
![4th-attempt-emotions2](wyniki\4th-attempt-emotions2.png)

***
# Podsumowanie

Najlepsze wyniki osiągałam przy mniejszych batch_size, więc najlepiej byłoby sprawdzić jakie pozostałe parametry dałyby dobre wyniki przy małym batch_size. Możliwe, że również większe max_length dla encodingów w tokenizerze dałoby lepsze wyniki. Wydaje mi się, że ostatecznie mój wybór modelu do dotrenowania nie wpłynął bardzo pozytywnie na wyniki, ale musiałabym to sprawdzić porównując z modelami trenowanymi na Herbercie. 

Za pomocą notatnika test_processing.ipynb wybrałam najlepszy z zapisanych modeli i za jego pomocą przewidziałam etykiety dla danych testowych. Nie spodziewam się jednak, żeby były one bardzo dobrze przez to jak słabo radziły sobie wszystkie wytrenowane modele.